<a href="https://colab.research.google.com/github/aadhamashraf/Intent-Based-Network-Generation-Augmentation/blob/main/laajresultsanalysis_converted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-

laajResultsAnalysis.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1C_xwLj0IEuy9Crp3Za7kDJskWmHLHitB
"""

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pprint

df = pd.read_excel("/content/gen_dataset_evaluation.xlsx" , sheet_name=1)
df.columns = df.columns.str.strip()
df["Intent Type"] = df["Intent Type"].str.strip()
df = df[df["Intent Type"] != ":---"]

df.columns

numericalValues  = ['Standards Alignment / Correctness (1-5)' , 'Complexity (1-5)' , 'Readability (1-5)' , 'Relevance (1-5)' , 'Coherence (1-5)' , 'Consistency (1-5)' , 'Completeness (1-5)' , 'Overall Score']

for i in numericalValues:
    df[i] = pd.to_numeric(df[i], errors='coerce')

df.info()

# **Low-Score Distribution by Evaluation Dimension**"""

In [ ]:
results = []

for col in numericalValues:
    total = df[col].notna().sum()
    below3 = (df[col] < 4).sum()
    perc = (below3 / total * 100) if total > 0 else 0
    results.append([col, below3, total, round(perc, 2)])

summary_df = pd.DataFrame(results, columns=["Column", "Count < 4", "Total", "Percentage %"])
print(summary_df)

all_summaries = []

for col in numericalValues:
    intent_counts = (df[df[col] < 4].groupby("Intent Type").size().reindex(df["Intent Type"].unique(), fill_value=0) )

    total = df[col].notna().sum()
    below4 = (df[col] < 4).sum()
    perc = (below4 / total * 100) if total > 0 else 0

    row = {"Metric": col, "Total <4": below4, "% <4": round(perc, 2)}
    row.update(intent_counts.to_dict())
    all_summaries.append(row)

intent_summary_df = pd.DataFrame(all_summaries)

intent_summary_df

bins = [0, 1, 2, 3, 4]
labels = ["0–1", "1–2", "2–3", "3–4"]

intent_summary_dfs = {}

for intent in df["Intent Type"].unique():
    all_summaries = []

    df_intent = df[df["Intent Type"] == intent]

    for col in numericalValues:
        total = df_intent[col].notna().sum()

        binned = pd.cut(df_intent[col], bins=bins, labels=labels, right=False)
        range_counts = binned.value_counts().reindex(labels, fill_value=0)
        range_perc = (range_counts / total * 100).round(2) if total > 0 else 0

        row = {"Metric": col, "Total": total}
        for label in labels:
            row[f"Count {label}"] = range_counts[label]
            row[f"% {label}"] = range_perc[label]

        all_summaries.append(row)

    intent_summary_dfs[intent] = pd.DataFrame(all_summaries)

deployment_summary = intent_summary_dfs.get('Deployment Intent')
deployment_summary

feasability_check_summary = intent_summary_dfs.get('Intent Feasibility Check')
feasability_check_summary

report_request_summary = intent_summary_dfs.get('Intent Report Request')
report_request_summary

modification_summary = intent_summary_dfs.get('Modification Intent')
modification_summary

regular_notification_request_summary = intent_summary_dfs.get('Regular Notification Request')
regular_notification_request_summary

performance_assurance_summary = intent_summary_dfs.get('Performance Assurance Intent')
performance_assurance_summary

# **Intent Types Analysis**"""

In [ ]:
justification_map = {
    "Standards Alignment / Correctness (1-5)": "Standards Justification",
    "Complexity (1-5)": "Complexity Justification",
    "Readability (1-5)": "Readability Justification",
    "Relevance (1-5)": "Relevance Justification",
    "Coherence (1-5)": "Coherence Justification",
    "Consistency (1-5)": "Consistency Justification",
    "Completeness (1-5)": "Completeness Justification",
    "Overall Score": "Overall Comments"
}

def get_intent_justification_details(df, intent_type, score_name):
    """
    Extract unique justifications for a given intent type and score/metric.

    Args:
        df (pd.DataFrame): The full DataFrame containing scores and justifications.
        intent_type (str): The intent type to filter (e.g., "Deployment Intent").
        score_name (str): The metric/score column name (e.g., "Complexity (1-5)").

    Returns:
        dict: Dictionary with binned ranges as keys.
              Each range contains unique justifications with their count and row indices.

    Example output:
    {
        '0–1': {
            'Justification A': {'count': 2, 'indices': [10, 25]},
            'Justification B': {'count': 1, 'indices': [42]}
        },
        '1–2': {...},
        '2–3': {...},
        '3–4': {...}
    }
    """
    df = df.copy()
    df.columns = df.columns.str.strip()
    df["Intent Type"] = df["Intent Type"].str.strip()

    df_intent = df[df["Intent Type"] == intent_type]

    justification_col = justification_map.get(score_name)
    if justification_col is None:
        raise ValueError(f"No justification column found for score '{score_name}'")

    binned = pd.cut(df_intent[score_name], bins=bins, labels=labels, right=False)

    details_per_range = {}
    for label in labels:
        subset = df_intent.loc[binned == label, justification_col].dropna()
        unique_justifications = subset.unique()
        just_detail = {}
        for just in unique_justifications:
            indices = subset[subset == just].index.tolist()
            just_detail[just] = {"count": len(indices), "indices": indices}
        details_per_range[label] = just_detail

    return details_per_range

## Intent Feasibility Check Analysis

### **Complexity Measure (1-5)**
"""

In [ ]:
feasibility_complexity = get_intent_justification_details(df, "Intent Feasibility Check", "Complexity (1-5)")
pprint.pprint(feasibility_complexity)

### **Standards Alignment / Correctness (1-5)**"""

In [ ]:
feasibility_standards_alignment = get_intent_justification_details(df, "Intent Feasibility Check", 'Standards Alignment / Correctness (1-5)')
pprint.pprint(feasibility_standards_alignment)

### **Readability (1-5)**"""

In [ ]:
feasibility_readability = get_intent_justification_details(df, "Intent Feasibility Check", 'Readability (1-5)')
pprint.pprint(feasibility_readability)

### **Relevance (1-5)**"""

In [ ]:
feasibility_relevance = get_intent_justification_details(df, "Intent Feasibility Check", 'Relevance (1-5)')
pprint.pprint(feasibility_relevance)

### **Coherence (1-5)**"""

In [ ]:
feasibility_coherence = get_intent_justification_details(df, "Intent Feasibility Check", 'Coherence (1-5)')
pprint.pprint(feasibility_coherence)

### **Consistency (1-5)**"""

In [ ]:
feasibility_consistency = get_intent_justification_details(df, "Intent Feasibility Check", 'Consistency (1-5)')
pprint.pprint(feasibility_consistency)

### **Completeness (1-5)**"""

In [ ]:
feasibility_completeness = get_intent_justification_details(df, "Intent Feasibility Check", 'Completeness (1-5)')
pprint.pprint(feasibility_completeness)

### **Overall Score (1-5)**"""

In [ ]:
feasibility_overall_score = get_intent_justification_details(df, "Intent Feasibility Check", 'Overall Score')
pprint.pprint(feasibility_overall_score)

## Intent Report Request Analysis

### **Complexity Measure (1-5)**
"""

In [ ]:
report_request_complexity = get_intent_justification_details(df, "Intent Report Request", "Complexity (1-5)")
pprint.pprint(report_request_complexity)

### **Standards Alignment / Correctness (1-5)**"""

In [ ]:
report_request_standards_alignment = get_intent_justification_details(df, "Intent Report Request", 'Standards Alignment / Correctness (1-5)')
pprint.pprint(report_request_standards_alignment)

### **Readability (1-5)**"""

In [ ]:
report_request_readability = get_intent_justification_details(df, "Intent Report Request", 'Readability (1-5)')
pprint.pprint(report_request_readability)

### **Relevance (1-5)**"""

In [ ]:
report_request_relevance = get_intent_justification_details(df, "Intent Report Request", 'Relevance (1-5)')
pprint.pprint(report_request_relevance)

### **Coherence (1-5)**"""

In [ ]:
report_request_coherence = get_intent_justification_details(df, "Intent Report Request", 'Coherence (1-5)')
pprint.pprint(report_request_coherence)

### **Consistency (1-5)**"""

In [ ]:
report_request_consistency = get_intent_justification_details(df, "Intent Report Request", 'Consistency (1-5)')
pprint.pprint(report_request_consistency)

### **Completeness (1-5)**"""

In [ ]:
report_request_completeness = get_intent_justification_details(df, "Intent Report Request", 'Completeness (1-5)')
pprint.pprint(report_request_completeness)

### **Overall Score (1-5)**"""

In [ ]:
report_request_overall_score = get_intent_justification_details(df, "Intent Report Request", 'Overall Score')
pprint.pprint(report_request_overall_score)

## Modification Intent Analysis

### **Complexity Measure (1-5)**
"""

In [ ]:
modification_complexity = get_intent_justification_details(df, "Modification Intent", "Complexity (1-5)")
pprint.pprint(modification_complexity)

### **Standards Alignment / Correctness (1-5)**"""

In [ ]:
modification_standards_alignment = get_intent_justification_details(df, "Modification Intent", 'Standards Alignment / Correctness (1-5)')
pprint.pprint(modification_standards_alignment)

### **Readability (1-5)**"""

In [ ]:
modification_readability = get_intent_justification_details(df, "Modification Intent", 'Readability (1-5)')
pprint.pprint(modification_readability)

### **Relevance (1-5)**"""

In [ ]:
modification_relevance = get_intent_justification_details(df, "Modification Intent", 'Relevance (1-5)')
pprint.pprint(modification_relevance)

### **Coherence (1-5)**"""

In [ ]:
modification_coherence = get_intent_justification_details(df, "Modification Intent", 'Coherence (1-5)')
pprint.pprint(modification_coherence)

### **Consistency (1-5)**"""

In [ ]:
modification_consistency = get_intent_justification_details(df, "Modification Intent", 'Consistency (1-5)')
pprint.pprint(modification_consistency)

### **Completeness (1-5)**"""

In [ ]:
modification_completeness = get_intent_justification_details(df, "Modification Intent", 'Completeness (1-5)')
pprint.pprint(modification_completeness)

### **Overall Score (1-5)**"""

In [ ]:
modification_overall_score = get_intent_justification_details(df, "Modification Intent", 'Overall Score')
pprint.pprint(modification_overall_score)

## Regular Notification Request Analysis
### **Complexity Measure (1-5)**
"""

In [ ]:
regular_notification_complexity = get_intent_justification_details(df, "Regular Notification Request", "Complexity (1-5)")
pprint.pprint(regular_notification_complexity)

### **Standards Alignment / Correctness (1-5)**"""

In [ ]:
regular_notification_standards_alignment = get_intent_justification_details(df, "Regular Notification Request", 'Standards Alignment / Correctness (1-5)')
pprint.pprint(regular_notification_standards_alignment)

### **Readability (1-5)**"""

In [ ]:
regular_notification_readability = get_intent_justification_details(df, "Regular Notification Request", 'Readability (1-5)')
pprint.pprint(regular_notification_readability)

### **Relevance (1-5)**"""

In [ ]:
regular_notification_relevance = get_intent_justification_details(df, "Regular Notification Request", 'Relevance (1-5)')
pprint.pprint(regular_notification_relevance)

### **Coherence (1-5)**"""

In [ ]:
regular_notification_coherence = get_intent_justification_details(df, "Regular Notification Request", 'Coherence (1-5)')
pprint.pprint(regular_notification_coherence)

### **Consistency (1-5)**"""

In [ ]:
regular_notification_consistency = get_intent_justification_details(df, "Regular Notification Request", 'Consistency (1-5)')
pprint.pprint(regular_notification_consistency)

### **Completeness (1-5)**"""

In [ ]:
regular_notification_completeness = get_intent_justification_details(df, "Regular Notification Request", 'Completeness (1-5)')
pprint.pprint(regular_notification_completeness)

### **Overall Score (1-5)**"""

In [ ]:
regular_notification_overall_score = get_intent_justification_details(df, "Regular Notification Request", 'Overall Score')
pprint.pprint(regular_notification_overall_score)

## Performance Assurance Intent Analysis

### **Complexity Measure (1-5)**
"""

In [ ]:
performance_assurance_complexity = get_intent_justification_details(df, "Performance Assurance Intent", "Complexity (1-5)")
pprint.pprint(performance_assurance_complexity)

### **Standards Alignment / Correctness (1-5)**"""

In [ ]:
performance_assurance_standards_alignment = get_intent_justification_details(df, "Performance Assurance Intent", 'Standards Alignment / Correctness (1-5)')
pprint.pprint(performance_assurance_standards_alignment)

### **Readability (1-5)**"""

In [ ]:
performance_assurance_readability = get_intent_justification_details(df, "Performance Assurance Intent", 'Readability (1-5)')
pprint.pprint(performance_assurance_readability)

### **Relevance (1-5)**"""

In [ ]:
performance_assurance_relevance = get_intent_justification_details(df, "Performance Assurance Intent", 'Relevance (1-5)')
pprint.pprint(performance_assurance_relevance)

### **Coherence (1-5)**"""

In [ ]:
performance_assurance_coherence = get_intent_justification_details(df, "Performance Assurance Intent", 'Coherence (1-5)')
pprint.pprint(performance_assurance_coherence)

### **Consistency (1-5)**"""

In [ ]:
performance_assurance_consistency = get_intent_justification_details(df, "Performance Assurance Intent", 'Consistency (1-5)')
pprint.pprint(performance_assurance_consistency)

### **Completeness (1-5)**"""

In [ ]:
performance_assurance_completeness = get_intent_justification_details(df, "Performance Assurance Intent", 'Completeness (1-5)')
pprint.pprint(performance_assurance_completeness)

### **Overall Score (1-5)**"""

In [ ]:
performance_assurance_overall_score = get_intent_justification_details(df, "Performance Assurance Intent", 'Overall Score')
pprint.pprint(performance_assurance_overall_score)

##Deployment Intent Analysis

### **Complexity Measure (1-5)**
"""

In [ ]:
deployment_complexity = get_intent_justification_details(df, "Deployment Intent", "Complexity (1-5)")
pprint.pprint(deployment_complexity)

### **Standards Alignment / Correctness (1-5)**"""

In [ ]:
deployment_standered_allignment= get_intent_justification_details(df, "Deployment Intent", 'Standards Alignment / Correctness (1-5)')
pprint.pprint(deployment_standered_allignment)

### **Readability (1-5)**"""

In [ ]:
deployment_readability = get_intent_justification_details(df, "Deployment Intent", "Readability (1-5)")
pprint.pprint(deployment_readability)

### **Relevance (1-5)**"""

In [ ]:
deployment_relevance = get_intent_justification_details(df, "Deployment Intent", "Relevance (1-5)")
pprint.pprint(deployment_relevance)

### **Coherence (1-5)**"""

In [ ]:
deployment_coherence = get_intent_justification_details(df, "Deployment Intent", "Choerence (1-5)")
pprint.pprint(deployment_coherence)

### **Consistency (1-5)**"""

In [ ]:
deployment_consistency = get_intent_justification_details(df, "Deployment Intent", "Consistency (1-5)")
pprint.pprint(deployment_consistency)

### **Completeness (1-5)**"""

In [ ]:
deployment_completeness = get_intent_justification_details(df, "Deployment Intent", "Completeness (1-5)")
pprint.pprint(deployment_completeness)

### **Overall Score (1-5)**"""

In [ ]:
deployment_overall_score = get_intent_justification_details(df, "Deployment Intent", "Overall Score")
pprint.pprint(deployment_overall_score)